In [126]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import re

In [119]:
def grab_labels(categories):
    #get top 10k addresses with labels
    #helper function
    def parse(res, c, url):
        html = requests.get(url).content
        soup = BeautifulSoup(html, 'html.parser')
        table = soup.find('tbody')
        table_rows = table.find_all('tr')
        for tr in table_rows:
            td = tr.find_all('td')
            if len(td) >2:
                row = [td[1].text.strip(),td[2].text.strip(), c]
                res.append(row)
        
        #how many pages?
        span = soup.find('span', style="padding: 2px 4px 4px 3px; border: 1px solid #D4D4D4; line-height: 30px; background-color: #EAEAEA; margin-top: 2px; height: 2px;")
        num_pages = int(span.find_all('b')[-1].decode_contents())
        
        return num_pages
    
    
    res = []
    for c in categories:
        #get first page, process
        print('processing {}'.format(c))
        url = 'https://etherscan.io/accounts/{}?ps=100&l={}'.format(1,c)
        num_pages = parse(res, c, url)
        time.sleep(3)
        if num_pages >1:
            for i in range(2, num_pages+1):
                url = 'https://etherscan.io/accounts/{}?ps=100&l={}'.format(i,c)
                _ = parse(res, c, url)
                time.sleep(3)
            
    df = pd.DataFrame(res, columns=['address', 'label', 'category'])
    
    return df

In [63]:
def get_urls():
    url ='https://etherscan.io/labelcloud'
    html = requests.get(url).content
    soup = BeautifulSoup(html, 'html.parser')
    links = soup.findAll("a", {"href" : re.compile('/accounts?l*')})
    hrefs = [l.get('href') for l in links]
    categories = []
    
    for h in hrefs:
        split = h.split(sep='=')
        if len(split) > 1:
            categories.append(split[1])
            
    return categories
    

In [66]:
categories = get_urls()

In [122]:
df_labels = grab_labels(categories)

processing 0x Ecosystem
processing 0xUniverse
processing Advertising
processing Bancor
processing Basic Attention Token
processing Binance
processing Blockchain Cuties
processing Bugs
processing Collectibles
processing Compound
processing Compromised
processing Crowdfunding
processing CryptoKitties
processing Dex
processing Donate
processing dYdX
processing Education
processing Electronics
processing ENS
processing Etheremon
processing Ethereum Foundation
processing Etheroll
processing Exchange
processing Fiat Gateway
processing Fomo3D
processing Gambling
processing Gods Unchained
processing Havven
processing Heist
processing High-Risk
processing Hyper Dragons
processing ICO Wallets
processing Loans
processing Maker
processing Marketplace
processing Media
processing Melon
processing Mining
processing Music
processing NFT
processing OTC
processing Others
processing Parity Bug
processing Phish/Hack
processing Poloniex
processing Prediction Market
processing Research
processing Spam Token

In [123]:
df_labels.shape

(4768, 3)

In [124]:
from google.cloud import bigquery
client = bigquery.Client()

In [127]:
df_labels.to_gbq(destination_table='alldata.etherscan_labelcloud', project_id='eth-tokens')

In [129]:
sql = '''
SELECT lc.*  FROM `eth-tokens.alldata.etherscan_labelcloud` lc
INNER JOIN `eth-tokens.alldata.etherscan` e
ON lc.address = e.address
'''

data = client.query(sql).to_dataframe()

In [131]:
data

,address,label,category
0,0x8d12a197cb00d4747a1fe03395095ce2a5cc6819,EtherDelta_2,Dex
1,0x2a0c0dbecc7e4d658f48e01e3fa353f44050c208,IDEX_1,Dex
2,0x04f062809b244e37e7fdc21d9409469c989c2342,Joyso,Dex
3,0x1ce7ae555139c5ef5a57cc8d814a867ee6ee33d8,TokenStore,Dex
4,0x000000000000000000000000000000000000dead,ENS-BurnAddress,ENS
5,0xbceaa0040764009fdcff407e82ad1f06465fd2c4,Bancor_Hack,Heist
6,0xf6884686a999f5ae6c1af03db92bab9c6d7dc8de,Coinrail_Hack,Heist
7,0x6ca33486eed915816560630b883a047c4e2b92df,Fake_MetadiumPresale,Heist
8,0xd4914762f9bd566bd0882b71af5439c0476d2ff6,GatecoinHack_1,Heist
9,0x1342a001544b8b7ae4a5d374e33114c66d78bd5f,GatecoinHack_2,Heist


In [134]:
sql = '''
SELECT lc.*  FROM `eth-tokens.alldata.etherscan_labelcloud` lc
INNER JOIN `eth-tokens.alldata.etherscan` e
ON lc.address = e.address
'''

data = client.query(sql).to_dataframe()

In [135]:
data

,address,label,category
0,0x8d12a197cb00d4747a1fe03395095ce2a5cc6819,EtherDelta_2,Dex
1,0x2a0c0dbecc7e4d658f48e01e3fa353f44050c208,IDEX_1,Dex
2,0x04f062809b244e37e7fdc21d9409469c989c2342,Joyso,Dex
3,0x1ce7ae555139c5ef5a57cc8d814a867ee6ee33d8,TokenStore,Dex
4,0x000000000000000000000000000000000000dead,ENS-BurnAddress,ENS
5,0xbceaa0040764009fdcff407e82ad1f06465fd2c4,Bancor_Hack,Heist
6,0xf6884686a999f5ae6c1af03db92bab9c6d7dc8de,Coinrail_Hack,Heist
7,0x6ca33486eed915816560630b883a047c4e2b92df,Fake_MetadiumPresale,Heist
8,0xd4914762f9bd566bd0882b71af5439c0476d2ff6,GatecoinHack_1,Heist
9,0x1342a001544b8b7ae4a5d374e33114c66d78bd5f,GatecoinHack_2,Heist
